# YOLOv3 模型技术文档

## 1. 整体架构
YOLOv3模型主要由以下几个部分组成:
- 主干网络(Backbone): DarkNet-53
- 特征金字塔网络(FPN)
- 检测头(Detection Head)
- 解码模块(Decode Box)

### 1.1 主干网络 DarkNet-53
DarkNet-53 采用残差结构,包含5个阶段的下采样,主要特点:

In [ ]:
# 输入图像经过处理后得到三个特征图:
- 52x52x256  (浅层特征,适合检测小目标)
- 26x26x512  (中层特征,适合检测中等目标) 
- 13x13x1024 (深层特征,适合检测大目标)

- 残差块结构:
    - 1x1 卷积降维
    - 3x3 卷积提取特征
    - 跳跃连接

### 1.2 特征金字塔网络
采用自顶向下的特征融合方式:
- 13x13特征图经过处理后上采样
- 与26x26特征图拼接
- 26x26特征图处理后上采样
- 与52x52特征图拼接

### 1.3 检测头
每个检测层输出格式:

In [ ]:
[batch_size, anchor_num * (5 + num_classes), height, width]

其中:
- 5表示: x,y,w,h,objectness 
- num_classes: 类别数

## 2. 关键实现细节
### 2.1 边界框解码

In [ ]:
def decode_box(self, inputs):
    # 1. 生成网格坐标
    grid_x = torch.linspace(0, input_width-1, input_width)
    grid_y = torch.linspace(0, input_height-1, input_height)
    
    # 2. 计算预测框
    pred_boxes[..., 0] = x.data + grid_x  # 中心x
    pred_boxes[..., 1] = y.data + grid_y  # 中心y
    pred_boxes[..., 2] = torch.exp(w.data) * anchor_w  # 宽
    pred_boxes[..., 3] = torch.exp(h.data) * anchor_h  # 高

### 2.2 非极大值抑制(NMS)

In [ ]:
def non_max_suppression(self, prediction,...):
    # 1. 转换预测框格式为 (x1,y1,x2,y2)
    # 2. 按置信度筛选
    conf_mask = (image_pred[:, 4] * class_conf[:, 0] >= conf_thres)
    
    # 3. 对每个类别分别进行NMS
    keep = nms(
        detections_class[:, :4],
        detections_class[:, 4] * detections_class[:, 5],
        nms_thres
    )

## 3. 使用说明
### 3.1 模型初始化

In [ ]:
anchors_mask = [[6,7,8], [3,4,5], [0,1,2]]
num_classes = 20  # 分类数量
model = YoloBody(anchors_mask, num_classes)

### 3.2 输入要求
- 输入图像尺寸: 416x416x3
- 图像预处理: 归一化至[0,1]

## 4. 模型运行建议
- 使用 CUDA 加速计算
- 批量处理提高吞吐量
- 根据实际应用调整 NMS 阈值
- 针对性调整 anchor 尺寸